In [1]:
#This is all 3 of our code for cleaning and feature engineering MERGED - hence the different CSVs we read, as we combined our work in this way

import pandas as pd

#Load the dataset, treating "(null)" as NaN for missing values
df = pd.read_csv("sqf-2023.csv", na_values="(null)", low_memory=False)

#Determine columns with more than 40% missing values and remove them
threshold = 0.4 * len(df)
columns_to_remove = df.columns[df.isnull().sum() > threshold]
df_cleaned = df.loc[:, df.isnull().sum() <= threshold]
df_cleaned.to_csv("sqf-2023-cleaned.csv", index=False)

print(f"columns removed({len(columns_to_remove)}):")
print(columns_to_remove.tolist())
print(len(df.columns))
print(len(df_cleaned.columns))
print(df_cleaned.columns.tolist())

#Reload cleaned dataset, replacing NaN with mean/mode values
df_cleaned = pd.read_csv("sqf-2023-cleaned.csv", na_values=["(null)", ""], low_memory=False)

for col in df_cleaned.columns:
    if pd.api.types.is_numeric_dtype(df_cleaned[col]):  # For numeric columns
        mean_value = df_cleaned[col].mean()
        df_cleaned[col].fillna(mean_value, inplace=True)
    else:  # For categorical columns
        mode = df_cleaned[col].mode()
        if not mode.empty:
            df_cleaned[col].fillna(mode[0], inplace=True)

df_cleaned.to_csv("sqf-2023-cleaned.csv", index=False)

#Check total number of missing cells
df = pd.read_csv("sqf-2023-cleaned.csv", na_values=["(null)", ""], low_memory=False)
total_null_or_empty = df.isnull().sum().sum()
print(f"Total '(null)' or empty cells: {total_null_or_empty}")

# Map 'Y'/'N' columns to 1/0 values
yes_no_columns = ['SUPERVISING_ACTION_CORRESPONDING_ACTIVITY_LOG_ENTRY_REVIEWED', 'OFFICER_EXPLAINED_STOP_FLAG',
                  'OTHER_PERSON_STOPPED_FLAG', 'SUSPECT_ARRESTED_FLAG', 'SUMMONS_ISSUED_FLAG',
                  'OFFICER_IN_UNIFORM_FLAG', 'FRISKED_FLAG', 'SEARCHED_FLAG', 'ASK_FOR_CONSENT_FLG',
                  'CONSENT_GIVEN_FLG', 'OTHER_CONTRABAND_FLAG', 'WEAPON_FOUND_FLAG',
                  'PHYSICAL_FORCE_VERBAL_INSTRUCTION_FLAG']

for col in df_cleaned.columns:
    if pd.api.types.is_numeric_dtype(df_cleaned[col]):
        mean_value = df_cleaned[col].mean()
        df_cleaned[col].fillna(mean_value, inplace=True)
        df_cleaned[col] = df_cleaned[col].round(2)
    elif col in yes_no_columns:
        df_cleaned[col] = df_cleaned[col].map({'Y': 1, 'N': 0})
    else:
        mode = df_cleaned[col].mode()
        if not mode.empty:
            df_cleaned[col].fillna(mode[0], inplace=True)

df_cleaned.to_csv("sqf-2023-final-cleaned.csv", index=False)

#Reload final cleaned data
df = pd.read_csv("sqf-2023-final-cleaned.csv", na_values=["(null)", ""], low_memory=False)
total_null_or_empty = df_cleaned.isnull().sum().sum()
print(f"Total '(null)' or empty cells: {total_null_or_empty}")

print(df_cleaned.info())       
print(df_cleaned.describe())   

#Add time of day categories based on STOP_FRISK_TIME
def categorize_time(time_str):
    if time_str == "00:00:00":  # Edge case for midnight
        return 'Night'
    hour = int(time_str.split(':')[0])
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df_cleaned['TIME_OF_DAY'] = df_cleaned['STOP_FRISK_TIME'].apply(categorize_time)

#Categorize stop duration into Short, Medium, or Long
def categorize_duration(duration):
    if duration <= 5:
        return 'Short'
    elif 6 <= duration <= 15:
        return 'Medium'
    else:
        return 'Long'

df_cleaned['STOP_DURATION_CATEGORY'] = df_cleaned['STOP_DURATION_MINUTES'].apply(categorize_duration)

#Convert height (feet) and weight (lbs) to metric units
def feet_to_meters(height_feet):
    return height_feet * 0.3048 

def lbs_to_kg(weight_lbs):
    return weight_lbs * 0.453592

df_cleaned['SUSPECT_WEIGHT_KG'] = df_cleaned['SUSPECT_WEIGHT'].apply(lbs_to_kg)
df_cleaned['SUSPECT_HEIGHT_M'] = df_cleaned['SUSPECT_HEIGHT'].apply(feet_to_meters)
df_cleaned['SUSPECT_BMI'] = (df_cleaned['SUSPECT_WEIGHT_KG'] / (df_cleaned['SUSPECT_HEIGHT_M'] ** 2)).round(2)

#Categorize BMI based on standard health ranges
def categorize_bmi(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif 18.5 <= bmi < 25:
        return 'Healthy Weight'
    elif 25 <= bmi < 30:
        return 'Overweight'
    elif 30 <= bmi < 35:
        return 'Class 1 Obesity'
    elif 35 <= bmi < 40:
        return 'Class 2 Obesity'
    else:
        return 'Class 3 Obesity (Severe Obesity)'

df_cleaned['BMI_CATEGORY'] = df_cleaned['SUSPECT_BMI'].apply(categorize_bmi)

#Map demeanor to numerical scores
demeanor_mapping = {
    'CALM': 1,
    'COOPERATIVE': 1,
    'EVASIVE': -1,
    'UPSET': -1,
    'YELLING': -2,
    'VIGILANT': 0
}

df_cleaned['DEMEANOR_SCORE'] = df_cleaned['DEMEANOR_OF_PERSON_STOPPED'].map(demeanor_mapping).fillna(0)

#Calculate compliance score
df_cleaned['COMPLIANCE_SCORE'] = (
    df_cleaned['DEMEANOR_SCORE'] +
    df_cleaned['CONSENT_GIVEN_FLG'] -
    df_cleaned['OTHER_CONTRABAND_FLAG']
)

df_cleaned.to_csv("sqf-2023-cleaned-with-features.csv", index=False)

#Filter and preprocess data further
data = pd.read_csv("sqf-2023-cleaned-with-features.csv", na_values=["(null)", ""], low_memory=False)
data = data[data['SUSPECTED_CRIME_DESCRIPTION'] != 'Other']  # Exclude 'Other' crimes

#One-hot encode specific columns
columns_to_encode = ['STOP_WAS_INITIATED', 'SUSPECT_SEX', 'SUSPECT_RACE_DESCRIPTION',
                     'STOP_LOCATION_BORO_NAME', 'STOP_WAS_INITIATED', 'SUSPECTED_CRIME_DESCRIPTION']
data_encoded = pd.get_dummies(data, columns=columns_to_encode, prefix=columns_to_encode)
data_encoded = data_encoded.applymap(lambda x: int(x) if isinstance(x, bool) else x)

#Calculate stop location total distance if coordinates are available
if 'STOP_LOCATION_X' in data.columns and 'STOP_LOCATION_Y' in data.columns:
    data_encoded['stop_total_distance'] = abs(data['STOP_LOCATION_Y'] - data['STOP_LOCATION_X'])
else:
    print("Columns 'STOP_LOCATION_X' and 'STOP_LOCATION_Y' not found in the dataset.")

#Group age into categories
bins = [0, 17, 30, 45, 60, 100]
labels = ['Under 18', '18-30', '31-45', '46-60', '60+']
data_encoded['AGE_GROUP'] = pd.cut(data_encoded['SUSPECT_REPORTED_AGE'], bins=bins, labels=labels, right=False)

#Create interaction feature
data_encoded['SEX_CRIME_INTERACTION'] = data_encoded['SUSPECT_SEX_FEMALE'].astype(str) + "_" + data_encoded['SUSPECTED_CRIME_DESCRIPTION_ASSAULT'].astype(str)

#One-hot encode new categorical features
data_encoded = pd.get_dummies(data_encoded, columns=['AGE_GROUP', 'TIME_OF_DAY'])
data_encoded = data_encoded.applymap(lambda x: int(x) if isinstance(x, bool) else x)

#Create new features combining flags
df_cleaned = data_encoded
df_cleaned['FRISKED_AND_ARRESTED'] = (df_cleaned['FRISKED_FLAG'] & df_cleaned['SUSPECT_ARRESTED_FLAG']).astype(int)

def combine_flags_and(frisked, arrested):
    if frisked and arrested:
        return 'Frisked and Arrested'
    elif frisked:
        return 'Frisked Only'
    elif arrested:
        return 'Arrested Only'
    else:
        return 'Neither'

df_cleaned['FRISKED_AND_ARRESTED_CAT'] = df_cleaned.apply(
    lambda row: combine_flags_and(row['FRISKED_FLAG'], row['SUSPECT_ARRESTED_FLAG']), axis=1
)

print(df_cleaned.columns)
rows_with_nan = df_cleaned[df_cleaned.isnull().any(axis=1)]
print(rows_with_nan)



columns removed(36):
['JURISDICTION_CODE', 'JURISDICTION_DESCRIPTION', 'OFFICER_NOT_EXPLAINED_STOP_DESCRIPTION', 'SUSPECT_ARREST_OFFENSE', 'SUMMONS_OFFENSE_DESCRIPTION', 'ID_CARD_IDENTIFIES_OFFICER_FLAG', 'SHIELD_IDENTIFIES_OFFICER_FLAG', 'VERBAL_IDENTIFIES_OFFICER_FLAG', 'FIREARM_FLAG', 'KNIFE_CUTTER_FLAG', 'OTHER_WEAPON_FLAG', 'PHYSICAL_FORCE_CEW_FLAG', 'PHYSICAL_FORCE_DRAW_POINT_FIREARM_FLAG', 'PHYSICAL_FORCE_HANDCUFF_SUSPECT_FLAG', 'PHYSICAL_FORCE_OC_SPRAY_USED_FLAG', 'PHYSICAL_FORCE_OTHER_FLAG', 'PHYSICAL_FORCE_RESTRAINT_USED_FLAG', 'PHYSICAL_FORCE_WEAPON_IMPACT_FLAG', 'BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG', 'BACKROUND_CIRCUMSTANCES_SUSPECT_KNOWN_TO_CARRY_WEAPON_FLAG', 'SUSPECTS_ACTIONS_CASING_FLAG', 'SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG', 'SUSPECTS_ACTIONS_DECRIPTION_FLAG', 'SUSPECTS_ACTIONS_DRUG_TRANSACTIONS_FLAG', 'SUSPECTS_ACTIONS_IDENTIFY_CRIME_PATTERN_FLAG', 'SUSPECTS_ACTIONS_LOOKOUT_FLAG', 'SUSPECTS_ACTIONS_OTHER_FLAG', 'SUSPECTS_ACTIONS_PROXIMITY_TO_SCEN

/var/folders/v6/3tplmktx41b84x1n38rsz4dw0000gn/T/ipykernel_33670/1325738382.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[col].fillna(mean_value, inplace=True)
/var/folders/v6/3tplmktx41b84x1n38rsz4dw0000gn/T/ipykernel_33670/1325738382.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

Total '(null)' or empty cells: 0


/var/folders/v6/3tplmktx41b84x1n38rsz4dw0000gn/T/ipykernel_33670/1325738382.py:62: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[col].fillna(mean_value, inplace=True)
/var/folders/v6/3tplmktx41b84x1n38rsz4dw0000gn/T/ipykernel_33670/1325738382.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

Total '(null)' or empty cells: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16971 entries, 0 to 16970
Data columns (total 46 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   STOP_ID                                                       16971 non-null  int64  
 1   STOP_FRISK_DATE                                               16971 non-null  object 
 2   STOP_FRISK_TIME                                               16971 non-null  object 
 3   YEAR2                                                         16971 non-null  int64  
 4   MONTH2                                                        16971 non-null  object 
 5   DAY2                                                          16971 non-null  object 
 6   STOP_WAS_INITIATED                                            16971 non-null  object 
 7   RECORD_STATUS_CODE                

/var/folders/v6/3tplmktx41b84x1n38rsz4dw0000gn/T/ipykernel_33670/1325738382.py:184: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_encoded = data_encoded.applymap(lambda x: int(x) if isinstance(x, bool) else x) #to get 1 hot encoding in 1 and 0
/var/folders/v6/3tplmktx41b84x1n38rsz4dw0000gn/T/ipykernel_33670/1325738382.py:206: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_encoded = data_encoded.applymap(lambda x: int(x) if isinstance(x, bool) else x)


Index(['STOP_ID', 'STOP_FRISK_DATE', 'STOP_FRISK_TIME', 'YEAR2', 'MONTH2',
       'DAY2', 'RECORD_STATUS_CODE', 'ISSUING_OFFICER_RANK',
       'ISSUING_OFFICER_COMMAND_CODE', 'SUPERVISING_OFFICER_RANK',
       ...
       'AGE_GROUP_18-30', 'AGE_GROUP_31-45', 'AGE_GROUP_46-60',
       'AGE_GROUP_60+', 'TIME_OF_DAY_Afternoon', 'TIME_OF_DAY_Evening',
       'TIME_OF_DAY_Morning', 'TIME_OF_DAY_Night', 'FRISKED_AND_ARRESTED',
       'FRISKED_AND_ARRESTED_CAT'],
      dtype='object', length=106)
       STOP_ID STOP_FRISK_DATE STOP_FRISK_TIME  YEAR2     MONTH2       DAY2  \
11789    11790      2023-09-13        20:55:00   2023  September  Wednesday   

      RECORD_STATUS_CODE ISSUING_OFFICER_RANK  ISSUING_OFFICER_COMMAND_CODE  \
11789                APP                  POM                            63   

      SUPERVISING_OFFICER_RANK  ...  AGE_GROUP_18-30  AGE_GROUP_31-45  \
11789                       LT  ...                1                0   

      AGE_GROUP_46-60  AGE_GROUP_60+  TI

In [2]:
df_cleaned = df_cleaned[df_cleaned['SUSPECT_BMI'].notna()]

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(df_cleaned.isnull().sum())

STOP_ID                                                                    0
STOP_FRISK_DATE                                                            0
STOP_FRISK_TIME                                                            0
YEAR2                                                                      0
MONTH2                                                                     0
DAY2                                                                       0
RECORD_STATUS_CODE                                                         0
ISSUING_OFFICER_RANK                                                       0
ISSUING_OFFICER_COMMAND_CODE                                               0
SUPERVISING_OFFICER_RANK                                                   0
SUPERVISING_OFFICER_COMMAND_CODE                                           0
SUPERVISING_ACTION_CORRESPONDING_ACTIVITY_LOG_ENTRY_REVIEWED               0
LOCATION_IN_OUT_CODE                                                       0

In [4]:
df_cleaned.to_csv("sqf-2023-cleaned-with-features-FINAL.csv", index=False)
print("Updated DataFrame saved to 'sqf-2023-cleaned-with-features-FINAL.csv'")

Updated DataFrame saved to 'sqf-2023-cleaned-with-features-FINAL.csv'
